In [1]:
import sys, os, datetime, requests
from mako.template import Template
sys.path.append("../../")

from utils.contract_utils import init_contract
from utils.network_utils import configure_network_and_connect

import brownie

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_style("whitegrid")

In [3]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network_and_connect(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

# Convex CRV locker

In [4]:
crv_depositor_contract = "0x8014595F2AB54cD7c604B00E9fb932176fDc86Ae"
crv_depositor_contract_obj = init_contract(crv_depositor_contract)
crv_depositor_contract_genesis_block = 12451019

In [5]:
current_block = brownie.web3.eth.block_number

In [6]:
def fetch_txes_for_contract(
    contract: str, 
    start: int,
    end: int,
    block_steps: int = 100000,
    etherscan_api_key: str = os.environ['ETHERSCAN_API_KEY']
):

    txes = Template("https://api.etherscan.io/api?module=account&action=txlist&address=${contract_addr}&startblock=${start_block}&endblock=${end_block}&sort=asc&page=1&offset=10000&apikey=${etherscan_key}")

    i = start
    contract_txes = []
    while i < end:

        etherscan_query = txes.render(
            etherscan_key=etherscan_api_key,
            start_block=i,
            end_block=i+block_steps,
            contract_addr=contract
        )
        etherscan_api_response = requests.post(etherscan_query)
        response_dict = etherscan_api_response.json()

        if not 'result' in response_dict.keys():
            break

        queried_txes = response_dict['result']
        if len(queried_txes) == 10000:  # probably missed txes: reduce steps and do it again.
            block_steps *= 0.8
            continue

        contract_txes.extend(queried_txes)
        i += block_steps

    return contract_txes

In [7]:
crv_deposit_txes = fetch_txes_for_contract(
    contract=crv_depositor_contract, 
    start=crv_depositor_contract_genesis_block,
    end=current_block,
    block_steps=100000,
    etherscan_api_key=os.environ['ETHERSCAN_API_KEY']
)

In [20]:
subset_crv_deposit_txes = crv_deposit_txes[-100:]

In [22]:
tx

{'blockNumber': '13296382',
 'timeStamp': '1632591693',
 'hash': '0xc474c6d85bc791a255b1c98fdb1b10557f84b4ee9b77fafb8568c0556897a770',
 'nonce': '1152',
 'blockHash': '0x99691314e055aee9d58d19141bf23aa5244633f19df58e77fb3185d174b1a130',
 'transactionIndex': '262',
 'from': '0x374f5fa69b977964d5242c95cd2d31db663bddf0',
 'to': '0x8014595f2ab54cd7c604b00e9fb932176fdc86ae',
 'value': '0',
 'gas': '350883',
 'gasPrice': '39867026188',
 'isError': '0',
 'txreceipt_status': '1',
 'input': '0x80ed71e40000000000000000000000000000000000000000000000281037585b81dfac9e00000000000000000000000000000000000000000000000000000000000000000000000000000000000000003fe65692bfcd0e6cf84cb1e7d24108e434a7587e',
 'contractAddress': '',
 'cumulativeGasUsed': '24077490',
 'gasUsed': '172260',
 'confirmations': '255'}

In [21]:
crv_lock_txes = {'timestamp': [], 'qt': []}
for tx in subset_crv_deposit_txes:
    try:
        tx_receipt = brownie.network.transaction.TransactionReceipt(tx['hash'])
        if tx_receipt.fn_name == 'deposit':
            crv_lock_txes['qt'].append(tx_receipt.events[0]['_value'] * 1e-18)
            crv_lock_txes['timestamp'].append(datetime.datetime.fromtimestamp(tx_receipt.timestamp))
    except:
        continue

df_crv_lock_txes = pd.DataFrame(crv_lock_txes)
df_crv_lock_txes.set_index('timestamp', inplace=True)
df_crv_lock_txes

,qt
timestamp,
2021-09-24 20:40:38,335.176000
2021-09-24 21:05:48,278.687823
2021-09-24 21:23:34,8372.078037
2021-09-24 21:34:48,5014.108750
2021-09-24 22:02:54,2000.000000
...,...
2021-09-25 20:52:22,8574.287600
2021-09-25 21:03:08,248.073336
2021-09-25 21:27:46,3904.747871


In [10]:
df_crv_lock_daily = pd.DataFrame()
df_crv_lock_daily['total_locked'] = df_crv_lock_txes.groupby(pd.Grouper(freq='1D'))['qt'].sum() / 10000
df_crv_lock_daily

,total_locked
timestamp,
2021-09-25,6.333903


# minted crv per day

In [19]:
release_schedule_file = "crv-released-locked-crv.csv"
df_daily_crv_minted = pd.read_csv(release_schedule_file)
df_daily_crv_minted = df_daily_crv_minted.drop(columns=['Total vote-locked CRV']).dropna()
df_daily_crv_minted

,DateTime,Total CRV
0,2020-08-13 22:17:28,2.302417e+01
1,2020-08-14 22:17:28,1.990220e+06
2,2020-08-15 22:17:28,3.980417e+06
3,2020-08-16 22:17:28,5.970614e+06
4,2020-08-17 22:17:28,7.960810e+06
...,...,...
504,2021-09-21 22:17:28,7.830796e+08
505,2021-09-22 22:17:28,7.845347e+08
506,2021-09-23 22:17:28,7.859897e+08
507,2021-09-24 22:17:28,7.874448e+08
